In [21]:
import adanet
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold


In [43]:
tf.__version__

'1.12.0'

In [23]:
data=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')
full=pd.concat([data,test])
full.drop(columns=['Cabin','PassengerId','Ticket'],inplace=True)
full.head()

/home/ec2-user/anaconda3/envs/adanet-env/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
0,22.0,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0
1,38.0,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0
2,26.0,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0
3,35.0,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0
4,35.0,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0


In [45]:
full.isnull().sum()

Age           0
Embarked      0
Fare          0
Name          0
Parch         0
Pclass        0
Sex           0
SibSp         0
Survived    418
Has_Age       0
Title         0
P_Ti_Age      0
dtype: int64

In [25]:
full['Sex']=full['Sex'].apply(lambda x : 1 if x=='male' else 0)
full=full[False==full['Embarked'].isnull()]

In [25]:
full['Embarked']=full['Embarked'].apply(lambda x : 0 if x=='S' else 1 if x=='C' else 2)
full['Fare']=full['Fare'].fillna(np.mean(full['Fare'].dropna().values))

## Transforming Fare to categorical 

In [26]:
fare_ranges = np.array([x*20 for x in range(30)])
full['Fare'] =fare_ranges.searchsorted(full.Fare)
full.head()

,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived
0,22.0,0,1,"Braund, Mr. Owen Harris",0,3,1,1,0.0
1,38.0,1,4,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,0,1,1.0
2,26.0,0,1,"Heikkinen, Miss. Laina",0,3,0,0,1.0
3,35.0,0,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,0,1,1.0
4,35.0,0,1,"Allen, Mr. William Henry",0,3,1,0,0.0


In [27]:
full['Has_Age'] = full['Age'].isnull().map(lambda x : 0 if x == True else 1)
# extracted title using name
full['Title'] = full.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
full.head()

,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Has_Age,Title
0,22.0,0,1,"Braund, Mr. Owen Harris",0,3,1,1,0.0,1,Mr
1,38.0,1,4,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,0,1,1.0,1,Mrs
2,26.0,0,1,"Heikkinen, Miss. Laina",0,3,0,0,1.0,1,Miss
3,35.0,0,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,0,1,1.0,1,Mrs
4,35.0,0,1,"Allen, Mr. William Henry",0,3,1,0,0.0,1,Mr


In [28]:
full['Title'] = full['Title'].replace(['Capt', 'Col', 'Countess', 'Don',
                                               'Dr', 'Dona', 'Jonkheer', 
                                                'Major','Rev','Sir'],'Unknown') 
full['Title'] = full['Title'].replace(['Mlle', 'Ms','Mme'],'Miss')
full['Title'] = full['Title'].replace(['Lady'],'Mrs')

In [29]:
full['Title'] = full['Title'].map({"Mr":0, "Unknown" : 1, "Master" : 2, "Miss" : 3, "Mrs" : 4 })
Pclass_title_pred = full.pivot_table(values='Age', index=['Pclass'], columns=['Title'],aggfunc=np.median).values
full['P_Ti_Age'] = full['Age']


In [30]:
# filling Missing age with Pclass & Title
for i in range(0,5):
    # 0,1,2,3,4
    for j in range(1,4):
        # 1,2,3
            full.loc[(full.Age.isnull()) & (full.Pclass == j) & (full.Title == i),'P_Ti_Age'] = Pclass_title_pred[j-1, i]
full['P_Ti_Age'] = full['P_Ti_Age'].astype('int')


### Transform Age to ranges

In [31]:
age_ranges = np.array([16,100])
full['Age'] =age_ranges.searchsorted(full.Age)

full['P_Ti_Age'] = age_ranges.searchsorted(full.P_Ti_Age)


In [32]:
full.head()

,Age,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Has_Age,Title,P_Ti_Age
0,1,0,1,"Braund, Mr. Owen Harris",0,3,1,1,0.0,1,0,1
1,1,1,4,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,0,1,1.0,1,4,1
2,1,0,1,"Heikkinen, Miss. Laina",0,3,0,0,1.0,1,3,1
3,1,0,3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,0,1,1.0,1,4,1
4,1,0,1,"Allen, Mr. William Henry",0,3,1,0,0.0,1,0,1


### Time for Ada-net

In [33]:
data_without_old_age=full.drop(columns=['Name','Has_Age','Age'])
data_without_old_age.head()

,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived,Title,P_Ti_Age
0,0,1,0,3,1,1,0.0,0,1
1,1,4,0,1,0,1,1.0,4,1
2,0,1,0,3,0,0,1.0,3,1
3,0,3,0,1,0,1,1.0,4,1
4,0,1,0,3,1,0,0.0,0,1


In [34]:
#min max normilize on fare 
column_to_normalize=['Fare']
data_without_old_age[column_to_normalize] = data_without_old_age[column_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))  

In [35]:
data_without_old_age = pd.get_dummies(data_without_old_age, columns=['Embarked','Sex','Parch','Pclass','Title','SibSp'])
data_without_old_age.head()

,Fare,Survived,P_Ti_Age,Embarked_0,Embarked_1,Embarked_2,Sex_0,Sex_1,Parch_0,Parch_1,...,Title_2,Title_3,Title_4,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,0.038462,0.0,1,1,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0.153846,1.0,1,0,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,0.038462,1.0,1,1,0,0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
3,0.115385,1.0,1,1,0,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0
4,0.038462,0.0,1,1,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0


In [36]:
to_predict=data_without_old_age[data_without_old_age['Survived'].isnull()]
data=data_without_old_age[False==data_without_old_age['Survived'].isnull()]

In [37]:
data.columns

Index(['Fare', 'Survived', 'P_Ti_Age', 'Embarked_0', 'Embarked_1',
       'Embarked_2', 'Sex_0', 'Sex_1', 'Parch_0', 'Parch_1', 'Parch_2',
       'Parch_3', 'Parch_4', 'Parch_5', 'Parch_6', 'Parch_9', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Title_0', 'Title_1', 'Title_2', 'Title_3',
       'Title_4', 'SibSp_0', 'SibSp_1', 'SibSp_2', 'SibSp_3', 'SibSp_4',
       'SibSp_5', 'SibSp_8'],
      dtype='object')

In [38]:
# Define the model head for computing loss and evaluation metrics.
head = tf.contrib.estimator.multi_class_head(n_classes=10)

# Feature columns define how to process examples.
feature_columns = [tf.feature_column.numeric_column(key = k) for k in list( data.drop(columns=['Survived']).columns.values)]# Learn to ensemble linear and neural network models.
estimator = adanet.AutoEnsembleEstimator(
    head=head,
    candidate_pool=[
        tf.contrib.estimator.LinearEstimator(
            head=head,
            feature_columns=feature_columns,
            optimizer=tf.train.FtrlOptimizer(learning_rate=0.01)),
         tf.contrib.estimator.DNNEstimator(
            head=head,
            feature_columns=feature_columns,
            optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.01),
            hidden_units=[1000, 500, 100])],
    max_iteration_steps=50)



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpnqy8_fmn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f44ee69b668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbi19_h92', '_tf_random_seed': None, '_save_summar

In [39]:
def parse_label_column(label_string_tensor):
    table = tf.contrib.lookup.index_table_from_tensor(tf.constant('Survived'))
    return table.lookup(label_string_tensor)


In [40]:
def input_fn(df):
    Features,labels = data,data.drop(columns=['Survived'])
    dataset = tf.data.Dataset.from_tensor_slices((dict(Features), labels))
    iterator = dataset.make_initializable_iterator()
    features, target = iterator.get_next()
    return features, parse_label_column(target)


In [41]:
type(feature_columns[0])

tensorflow.python.feature_column.feature_column._NumericColumn

In [ ]:
estimator.train(input_fn=lambda: input_fn(data), steps=100)

In [ ]:
metrics = estimator.evaluate(input_fn=input_fn_eval, steps=10)
predictions = estimator.predict(input_fn=input_fn("predict", training=False, batch_size=1))
